In [1]:
!pwd

/home/pafrank/Documents/coding/chess-position-embedding/test


### Test pgn_extract.py tool

In [3]:
!/home/pafrank/anaconda3/envs/pos-emb/bin/python ../tools/pgn_extract.py --help

usage: pgn_extract.py [-h] [--save_position SAVE_POSITION] [--tuples TUPLES]
                      [--save_tuples SAVE_TUPLES] [--chunksize CHUNKSIZE]
                      [--filter KEY1=VAL1]
                      input

Generate bitboards and training samples

positional arguments:
  input                 pgn file with input games

optional arguments:
  -h, --help            show this help message and exit
  --save_position SAVE_POSITION
                        h5py file to store the encoded positions
  --tuples TUPLES       Do you want to generate tuples as well?
                        True|False(default)
  --save_tuples SAVE_TUPLES
                        h5py file to store the encoded tuples
  --chunksize CHUNKSIZE
                        Chunk size for paginating games
  --filter KEY1=VAL1    Filter out games. Options: time_min, elo_min. Usage:
                        --filter key1=val1 --filter key2=val2


In [2]:
# convert all games in that file
# .pgn file ending does not need to be provided, it is automatically added, same for output(.h5 file)
# do not generate tuples for training
!time /home/pafrank/anaconda3/envs/pos-emb/bin/python ../tools/pgn_extract.py ../data/raw/test --save_position ../data/bitboards/test-bb1 --chunksize 10000

Input file at: ../data/raw/test
Filter options: {}
Positions saved at: ../data/bitboards/test-bb1
Tuples generated: False
Tuples saved at: None
Chunksize: 10000



Game positions saved.
Chunk 0 processed.
3.68user 0.07system 0:03.78elapsed 99%CPU (0avgtext+0avgdata 67816maxresident)k
2616inputs+224outputs (259major+15037minor)pagefaults 0swaps


In [4]:
# convert all games in that file
# .pgn file ending does not need to be provided, it is automatically added, same for output(.h5 file)
# also generate tuples for training
# this time filter games based on elo and time control
!time /home/pafrank/anaconda3/envs/pos-emb/bin/python ../tools/pgn_extract.py ../data/raw/test --save_position ../data/bitboards/test-bb2 --chunksize 10000 --tuples True --save_tuples ../data/train_small/test2-tuples-strong --filter elo_min=1800 --filter time_min=61

Input file at: ../data/raw/test
Filter options: {'elo_min': '1800', 'time_min': '61'}
Positions saved at: ../data/bitboards/test-bb2
Tuples generated: True
Tuples saved at: ../data/train_small/test2-tuples-strong
Chunksize: 10000



Game positions saved.
Tuples generated.
Tuples saved.
Chunk 0 processed.
1.14user 0.05system 0:01.23elapsed 97%CPU (0avgtext+0avgdata 47536maxresident)k
192inputs+80outputs (20major+11616minor)pagefaults 0swaps


### Test index_from_bitboards.py tool

In [1]:
!/home/pafrank/anaconda3/envs/pos-emb/bin/python ../tools/index_from_bitboards.py --help

usage: index_from_bitboards.py [-h] [--table_key TABLE_KEY]
                               [--save_path SAVE_PATH]
                               input_directory

Generate a searchable faiss index from bitboards.

positional arguments:
  input_directory       directory that contains all bitboards (stored in h5
                        files), which should be stored in an index

optional arguments:
  -h, --help            show this help message and exit
  --table_key TABLE_KEY
                        substring, that is contained in all relevant h5 table
                        names (default: position should work out of the box)
  --save_path SAVE_PATH
                        full path- and filename to the output index, defaults
                        to the input directory


In [2]:
# using default
!time /home/pafrank/anaconda3/envs/pos-emb/bin/python ../tools/index_from_bitboards.py ../data/bitboards/testdir

Input directory: ../data/bitboards/testdir
Table key for h5 tables: position
Index saved at: 


Files to be added to index:
['/home/pafrank/Documents/coding/chess-position-embedding/data/bitboards/testdir/lichess_db_standard_rated_2013-03-bb.h5']
File /home/pafrank/Documents/coding/chess-position-embedding/data/bitboards/testdir/lichess_db_standard_rated_2013-03-bb.h5 has keys <KeysViewHDF5 ['game_id_0', 'game_id_1', 'position_0', 'position_1']>
The index contains 10.857 million positions.
Index successfully saved at /home/pafrank/Documents/coding/chess-position-embedding/data/bitboards/testdir/bitboard_index.
66.96user 11.14system 1:18.69elapsed 99%CPU (0avgtext+0avgdata 3394124maxresident)k
88896inputs+2056984outputs (0major+825816minor)pagefaults 0swaps


In [5]:
!time /home/pafrank/anaconda3/envs/pos-emb/bin/python ../tools/index_from_bitboards.py ../data/bitboards/testdir --table_key position_0 --save_path ../data/test_index2

Input directory: ../data/bitboards/testdir
Table key for h5 tables: position_0
Index saved at: ../data/test_index2


Files to be added to index:
['/home/pafrank/Documents/coding/chess-position-embedding/data/bitboards/testdir/lichess_db_standard_rated_2013-03-bb.h5']
File /home/pafrank/Documents/coding/chess-position-embedding/data/bitboards/testdir/lichess_db_standard_rated_2013-03-bb.h5 has keys <KeysViewHDF5 ['game_id_0', 'game_id_1', 'position_0', 'position_1']>
The index contains 6.843 million positions.
Index successfully saved at /home/pafrank/Documents/coding/chess-position-embedding/data/test_index2.
42.35user 3.83system 0:46.18elapsed 100%CPU (0avgtext+0avgdata 2930068maxresident)k
16inputs+1296560outputs (0major+414575minor)pagefaults 0swaps


In [1]:
!/home/pafrank/anaconda3/envs/pos-emb/bin/python ../tools/train_model.py -h

2020-04-20 16:38:39.572134: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory
2020-04-20 16:38:39.572684: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory
2020-04-20 16:38:39.572701: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
usage: train_model.py [-h] config

Train a chess position embedding with tensorflow.

positional arguments:
  config      json config file to read settings from

optional arguments:
  -h, --help  show this help message and exit


In [2]:
!/home/pafrank/anaconda3/envs/pos-emb/bin/python ../tools/model_inference.py -h

2020-04-20 19:46:59.674446: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory
2020-04-20 19:46:59.674911: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory
2020-04-20 19:46:59.674935: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
usage: model_inference.py [-h] [--table_prefix TABLE_PREFIX]
                          [--embedding_table_prefix EMBEDDING_TABLE_PREFIX]
                          [--batch_size BATCH_SIZE] [--float16 FLOAT16]
                          model_dir bitboard_dir

Infer bitboard embeddi

In [3]:
!/home/pafrank/anaconda3/envs/pos-emb/bin/python ../tools/index_from_embedding.py -h

2020-04-20 20:07:31.967074: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory
2020-04-20 20:07:31.967548: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory
2020-04-20 20:07:31.967573: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
usage: index_from_embedding.py [-h] [--table_prefix TABLE_PREFIX]
                               [--save_path SAVE_PATH] [--chunks CHUNKS]
                               [--train_frac TRAIN_FRAC]
                               index_factory_string embedding_dir

Generate a searcha